<a href="https://colab.research.google.com/github/JoaoADuarte/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install transformers datasets peft

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,000 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [ ]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Configurar o ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

# Simular um navegador real com cabeçalhos personalizados
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")
driver = webdriver.Chrome(options=chrome_options)

conn = sqlite3.connect("chatbot_jardinagem_db")
cursor = conn.cursor()

# Nossa tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS artigos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fonte TEXT,
    titulo TEXT,
    conteudo TEXT,
    tags TEXT
);
""")
conn.commit()
print("DB criado!")


# Insere dados no BD
def inserir_dados(fonte, titulo, conteudo, tags):
  cursor.execute("""
  INSERT INTO artigos (fonte, titulo, conteudo, tags)
  VALUES (?,?,?,?)
  """, (fonte, titulo, conteudo, tags))
  conn.commit()

# Busca os dados do nosso DB
def buscar_conteudo(palavra_chave):
    cursor.execute("""
    SELECT titulo, conteudo FROM artigos
    WHERE tags LIKE ? OR conteudo LIKE ?
    """, (f"%{palavra_chave}%", f"%{palavra_chave}%"))
    resultados = cursor.fetchall()
    return resultados



# URLs das páginas para scraping
urls = [
    "https://www.rhs.org.uk/vegetables/tomatoes/grow-your-own",
    "https://www.almanac.com/plant/tomatoes",
    "https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html",
    "https://www.petz.com.br/blog/plantas/como-plantar-tomate/"
]

for url in urls:
    driver.get(url)
    time.sleep(3)  # Esperar a página carregar


    # Forçando atualização da página para evitar que reutilize dados
    driver.refresh()
    time.sleep(3)
    print(f"\n Extraindo dados de: {url}")

    # Título do artigo, onde pega o "h1" do HTML como título
    try:
      titulo_element = driver.find_element(By.TAG_NAME, "h1")
      titulo = titulo_element.text
    except:
      titulo = "Título não encontrado"

    # Extrair texto principal do site (ajustar conforme necessário)
    elementos_texto_p = driver.find_elements(By.TAG_NAME, "p")  # Pegar todos os parágrafos
    conteudo = "\n".join([e.text for e in elementos_texto if e.text.strip()])

    # Mostrar trecho do conteúdo extraído
    print(f"{titulo} - {conteudo[:300]}...\n")

    # Gerar tags com base no título de forma automática
    tags = ", ".join(titulo.lower().split())

    # Inserir dados no BD
    inserir_dados(url, titulo, conteudo, tags)
    print(f"Dados inseridos com sucesso em {url}")

# Fechar o navegador
driver.quit()

# Testando a busca
palavra_chave = "Tomatoes"
resultados = buscar_conteudo(palavra_chave)

# Exibir os primeiros 500 caracteres de cada site
print("\nResultados da busca:\n")
for titulo, conteudo in resultados:
    print(f"{titulo}\n{conteudo[:500]}...\n")  # Mostra um trecho do conteúdo

# Fechar a conexão com o banco de dados
conn.close()

✅ Banco de dados pronto!

🔍 Extraindo dados de: https://www.rhs.org.uk/vegetables/tomatoes/grow-your-own
📌 How to grow tomatoes - Our detailed growing guide will help you with each step in successfully growing Tomatoes.
Cherry tomatoes are sweet and delicious, especially when harvested from your own plants
Tomatoes are one of the most popular crops to grow – they can be started from seed indoors or bought as young plants, then...

✅ Dados inseridos com sucesso: https://www.rhs.org.uk/vegetables/tomatoes/grow-your-own

🔍 Extraindo dados de: https://bonnieplants.com/blogs/how-to-grow/growing-tomatoes
📌 GROWING TOMATOES - Free Shipping on orders over $50!
Free Shipping on orders over $50!
Free Shipping on orders over $50!
Free shipping on orders over $50
Learn how to grow tomatoes. Get tips about growing tomatoes in the ground, raised beds, or containers. Care for tomato plants from planting to harvesting.
Choosing t...

✅ Dados inseridos com sucesso: https://bonnieplants.com/blogs/how-to

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


modelo_nome = "google/flan-t5-large"

# Carregar o modelo e o tokenizador
tokenizer = T5Tokenizer.from_pretrained(modelo_nome)
model = T5ForConditionalGeneration.from_pretrained(modelo_nome)


 # Criação da função para pré-processamento das perguntas, alimentar o FLAN-T5 e gerar uma resposta.
def gerar_resposta(pergunta, contexto):
    prompt = f"""
    You are an expert in gardening..
    Answer the question clearly and objectively based only on the context below:

    Context:
    {contexto}

    Question: {pergunta}
    Question:
    """

    # Tokenização correta com o prompt completo
    inputs = tokenizer(prompt, return_tensors="pt")

    # Gerar a resposta com temperatura controlada para evitar repetições
    resposta_ids = model.generate(
        **inputs,
        max_length=256,  # Aumentar o comprimento máximo da resposta
        max_new_tokens=512,
        num_beams=16,     # Usar mais "beams" para melhorar a busca
        temperature=0.7, # Controlar a criatividade (valores menores = mais determinístico)
        top_p=0.9,       # Usar amostragem com nucleus sampling
        early_stopping=True,
        no_repeat_ngram_size=2  # Evitar repetições de 3+ palavras
    )
    # Decodificar a resposta
    resposta = tokenizer.batch_decode(resposta_ids[0], skip_special_tokens=True)


    return resposta

def buscar_e_responder(pergunta):
    # Conectar ao banco de dados
    conn = sqlite3.connect("chatbot_jardinagem_db")
    cursor = conn.cursor()

    # Dividir a pergunta em palavras-chave
    palavras_chave = pergunta.lower().split()

    # Criar a consulta SQL mais eficiente
    query = """
    SELECT titulo, conteudo FROM artigos
    WHERE conteudo LIKE ?
    ORDER BY LENGTH(conteudo) ASC
    LIMIT 3
    """
    valores = [f"%{pergunta}%"]  # Buscar a pergunta completa primeiro

    cursor.execute(query, valores)
    resultados = cursor.fetchall()

    conn.close()

    if not resultados:
        return "Não encontrei informações suficientes sobre isso."

    # Selecionar os 500 primeiros caracteres do melhor resultado
    contexto = "\n".join([f"{titulo}: {conteudo[:500]}" for titulo, conteudo in resultados])

    # Gerar resposta com FLAN-T5
    resposta = gerar_resposta(pergunta, contexto)

    return resposta



pergunta = "how to plant tomatoes?"
resposta = buscar_e_responder(pergunta)
print("🤖 Chatbot:", resposta)

🤖 Chatbot: Não encontrei informações suficientes sobre isso.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

inputs = tokenizer("how to plant tomatoes?:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))



['Place a sprig of tomato in a pot and cover with a layer of']
